In [1]:
# import Sent2textDataset
import torch
from sklearn.metrics import precision_score
from torch.utils.data import DataLoader


import sys
sys.path.append("/data/hdd1/brain/BraTS19/YandexCup/ru-clip")

import pandas as pd
from clip.evaluate.utils import (
    load_weights_only,
    show_similarity,
)
from sklearn.metrics import f1_score
from CustomDataset import Sent2textDataset
model, args = load_weights_only("ViT-B/32-small",seq_length = 15)

/home/viktor.kumpan/.miniconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
Some weights of the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import numpy as np
from torch.utils.data import SubsetRandomSampler
def sempler(data_train, batch_size = 4,split = .8):
    
    data_size = len(data_train)

    validation_split = split
    split = int(np.floor(validation_split * data_size))
    indices = list(range(data_size))
    np.random.shuffle(indices)

    train_indices,val_indices = indices[split:],indices[:split]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    

    train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                              sampler=train_sampler,)
    
    val_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                            sampler=val_sampler,)

    return train_loader,val_loader



In [79]:
n_classes = 5

def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):
    loss_history = []
    train_history = []
    val_history = []
    val_loss_hist = []
    metric_y_val = metric_p_val = None
    
#     scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
#                                                   T_0=40, 
#                                                   T_mult=2,
#                                                   eta_min=1e-9)
    for epoch in range(num_epochs):
        
        print(epoch)
        model.train()
        
        correct_samples = 0
        total_samples = 0
        loss_accum = 0
        for i_step, data  in enumerate(train_loader):
            
            
                imgs_gpu = torch.squeeze(data[0].cuda(),1)
                texts_gpu = data[1].cuda()
                att_mask_gpu = data[2].cuda()
                label_gpu = torch.arange(data[0].shape[0]).cuda()
                
                
                _, prediction = model(img_input={"x": imgs_gpu},
                                    text_input={"x": texts_gpu, "attention_mask":att_mask_gpu})
                
            
                loss_value = loss(prediction, label_gpu)
                
                _, preds = torch.max(prediction, 1)
                
                print(preds)
                

                optimizer.zero_grad()
                loss_value.backward()
                optimizer.step()
                
                
                if i_step == 0 and epoch == 0:
                    metric_y = label_gpu.cpu().numpy()
                    metric_p = preds.cpu().numpy()
                else:
                    metric_y = np.concatenate((metric_y, label_gpu.cpu().numpy()))
                    metric_p = np.concatenate((metric_p, preds.cpu().numpy())) 
                    
                correct_samples += torch.sum(preds == label_gpu)
                loss_accum += loss_value
                total_samples += label_gpu.shape[0]
                
                del imgs_gpu
                del texts_gpu
                del att_mask_gpu
            
                del label_gpu
        
        
        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = correct_samples / total_samples
#         writer.add_scalar("Loss/train", ave_loss, epoch)
#         writer.add_scalar("Acc/train", train_accuracy, epoch)
#         writer.add_scalar("F1/train", f1_score(metric_y,metric_p), epoch)

        val_accuracy, loss_val,metric_y_val, metric_p_val = compute_valid(model, val_loader, loss, epoch,metric_y_val, metric_p_val)
#         writer.add_scalar("Loss/valid", loss_val, epoch)
#         writer.add_scalar("Acc/valid", val_accuracy, epoch)
#         writer.add_scalar("F1/valid", f1_score(metric_y_val,metric_p_val), epoch)
        
#         writer.add_scalar("Lr/epoch", scheduler.get_last_lr()[-1], epoch)
#         scheduler.step(epoch)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        val_loss_hist.append(loss_val)
        

        print("Average loss: %f, Val loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss,loss_val, train_accuracy, val_accuracy))
    return metric_y_val, metric_p_val
#         print('Epoch:', epoch, 'LR:', scheduler.get_last_lr())

def compute_valid(model, loader, loss,epoch, metric_y= None, metric_p=None):
    model.eval()
    with torch.no_grad():
        correct_samples = 0
        total_samples = 0
        loss_accum = 0
        
        for i_step, data in enumerate(loader):
            
            imgs_gpu = torch.squeeze(data[0].cuda(),1)
            texts_gpu = data[1].cuda()
            att_mask_gpu = data[2].cuda()
            label_gpu = torch.arange(data[0].shape[0]).cuda()

            _, prediction = model(img_input={"x": imgs_gpu},
                                    text_input={"x": texts_gpu, "attention_mask":att_mask_gpu})
            
            
            loss_value = loss(prediction, label_gpu)
            _, preds = torch.max(prediction, 1)
            print(preds.cpu().numpy().tolist())
            
            if i_step == 0 and epoch == 0:
                metric_y = label_gpu.cpu().numpy()
                metric_p = preds.cpu().numpy()
            else:
                metric_y = np.concatenate((metric_y, label_gpu.cpu().numpy()))
                metric_p = np.concatenate((metric_p, preds.cpu().numpy())) 
            
            
            correct_samples += torch.sum(preds == label_gpu)
            total_samples += label_gpu.shape[0]
            loss_accum += loss_value

            del imgs_gpu
            del texts_gpu
            del att_mask_gpu
            
            del label_gpu
                
        loss_val = loss_accum / (i_step + 1)
        val_accuracy = correct_samples / total_samples
        return val_accuracy, loss_val, metric_y, metric_p

In [4]:
path_t_csv = pd.read_csv("data/60k_data_preproc.csv")[:2500] #
path_i_json = "data/images.json"
path_i_folder = "data/images"


# в csv file для каждого касса должен быть массив


ds = Sent2textDataset(path_t_csv,
                      path_i_json,path_i_folder,
                      down_data = False,
                      n_classes = 5,args = args)

5462418it [00:12, 445537.07it/s]


Broken 0 images
2278 images in folder from 2500 in csv file


In [7]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [8]:
model = model.cuda().float()
for name,param in model.named_parameters():
    if name.find("visual_encoder") != -1:
        param.requires_grad = False
    if name.find("text_encoder")!= -1:
        param.requires_grad = False
        
    if name.find("11")!=-1 and name.find("visual_encoder")== -1:
        param.requires_grad = True
        
    if name.find("projection")!=-1:
        param.requires_grad = True
    if name == "logit_scale":
        param.requires_grad = True
        
    
[name for name,param in model.named_parameters() if param.requires_grad ]

['logit_scale',
 'visual_encoder.projection.linear1.weight',
 'visual_encoder.projection.linear2.weight',
 'visual_encoder.projection.layer_norm.weight',
 'visual_encoder.projection.layer_norm.bias',
 'text_encoder.model.h.11.ln_1.weight',
 'text_encoder.model.h.11.ln_1.bias',
 'text_encoder.model.h.11.attn.c_attn.weight',
 'text_encoder.model.h.11.attn.c_attn.bias',
 'text_encoder.model.h.11.attn.c_proj.weight',
 'text_encoder.model.h.11.attn.c_proj.bias',
 'text_encoder.model.h.11.ln_2.weight',
 'text_encoder.model.h.11.ln_2.bias',
 'text_encoder.model.h.11.mlp.c_fc.weight',
 'text_encoder.model.h.11.mlp.c_fc.bias',
 'text_encoder.model.h.11.mlp.c_proj.weight',
 'text_encoder.model.h.11.mlp.c_proj.bias',
 'text_encoder.projection.linear1.weight',
 'text_encoder.projection.linear2.weight',
 'text_encoder.projection.layer_norm.weight',
 'text_encoder.projection.layer_norm.bias']

In [9]:
loss = CrossEntropyLoss()
optimizer = Adam([param for param in model.parameters() if param.requires_grad], lr = 1e-04, weight_decay = 1e-05)


In [10]:
train_dl,val_dl =  sempler(ds, batch_size=20)

In [80]:
result = train_model(model, train_dl, val_dl, loss, optimizer, 64)

0
tensor([ 0,  3,  2,  3,  4,  5, 15,  7,  8,  9, 17, 19, 12, 13, 13, 17, 16, 15,
        19, 16], device='cuda:0')
tensor([19, 16,  2,  3,  4, 19,  6,  7,  8,  9, 15,  5, 12, 13,  1, 15,  6, 17,
         0, 14], device='cuda:0')
tensor([ 0,  1,  2,  4,  4,  3,  6,  7,  6,  9,  1, 11, 12, 12,  4,  0, 16, 17,
        12, 19], device='cuda:0')
tensor([ 4,  1,  3,  3,  4,  3,  6,  9,  8,  7, 10, 11, 17, 13, 14, 15, 17, 17,
        18,  1], device='cuda:0')
tensor([ 0,  1,  2,  3,  4,  5,  6,  4,  8, 18, 10, 11,  8,  8,  0, 15, 16, 17,
        18, 15], device='cuda:0')
tensor([ 0, 11,  5,  3,  4,  5,  6, 17,  8, 15, 10, 11, 12, 13, 14, 15, 12, 17,
        12, 19], device='cuda:0')
tensor([ 0,  1,  2,  3,  4, 11,  6, 15,  8, 18, 10, 11, 12, 13,  3, 12, 15, 17,
        18,  1], device='cuda:0')
tensor([14,  1,  2,  3,  5,  5, 11,  7, 19,  9, 10, 11, 12,  6, 14, 15,  1, 12,
        19, 19], device='cuda:0')
tensor([13,  1, 13,  3,  0,  4,  6,  7,  8, 13,  6, 11, 12, 14, 14, 15, 16, 17,
      

KeyboardInterrupt: 

In [78]:
np.arange(5).to_list()

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'